In [1]:
import selenium
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import time

1. Write a python program which searches all the product under a particular product from www.amazon.in. 
The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search 
for guitars.

In [2]:
amz_driver = webdriver.Chrome("D:\c")

In [3]:
amz_driver.get('https://www.amazon.in/')
time.sleep(2)

In [4]:
search_button = amz_driver.find_element(By.XPATH, "/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[2]/div[1]/input")
user_input = input("Please search your product: ")
p_search = user_input+'s' #use concatination to complete the search word 
search_button.send_keys(p_search)
time.sleep(2)

submit_button = amz_driver.find_element(By.XPATH, "/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[3]/div/span/input")
submit_button.click()
time.sleep(2)

Please search your product: guitar


2. In the above question, now scrape the following details of each product listed in first 3 pages of your 
search results and save it in a data frame and csv. In case if any product has less than 3 pages in search 
results then scrape all the products available under that product name. Details to be scraped are: "Brand 
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and 
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [5]:
#creating empty list to store fetched data
brand = []
Name_of_product = []
price = []
expected_delivery = []
product_url = []
brands = [] #create this empty list to split the brand data and store only brand name

start = 0
end = 3

amz_driver.refresh()
time.sleep(3)
for i in range (start, end): # loop use to jump on next page
    

    brand_name =amz_driver.find_elements(By.XPATH, "//span[@class = 'a-size-base-plus a-color-base a-text-normal']")
    for i in brand_name:
        brand.append(i.text)
        for i in brand:
            brands.append(i.split(" ")[0])
        
    prices = amz_driver.find_elements(By.XPATH, "//span[@class = 'a-price-whole']")
    for i in prices:
        price.append(i.text)
        
    expected_d = amz_driver.find_elements(By.XPATH, "//span[@class = 'a-color-base a-text-bold']")
    for i in expected_d:
        expected_delivery.append(i.text)
        
    p_url = amz_driver.find_elements(By.XPATH, "//h2[@class = 'a-size-mini a-spacing-none a-color-base s-line-clamp-4']/a")
    for i in p_url:
        product_url.append(i.get_attribute('href'))
        
    time.sleep(2)   #to make sure all the data fetched on time 
    next_page = amz_driver.find_element(By.XPATH, "//a[@class = 's-pagination-item s-pagination-next s-pagination-button s-pagination-separator']")
    next_page.click()
    time.sleep(2) #to make sure new page upload successfully

#printing on lenght to check inequality in the fetched data
print(len(price))    
print(len(expected_delivery))
print(len(product_url))
print(len(brands))

190
190
180
17020


In [6]:
products_attributes_lists = [price, brands, expected_delivery, product_url]
#created a nested list to stor all the data so that we can create DataFrame with uneqal data.

In [7]:
p_df1 = pd.DataFrame(products_attributes_lists)
#print(p_df1)
p_df2 = p_df1.transpose() # use transpose to convert rows into columns
#print(p_df2)
p_df3 = pd.DataFrame(p_df2)
p_df3.columns = ['Price', 'Brand Name', 'Expected Delivery', 'Prouduct link']
p_df3


,Price,Brand Name,Expected Delivery,Prouduct link
0,"6,299",Kadence,"Tuesday, December 27",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,"9,900",Kadence,"Thursday, December 29",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,"4,949",Kadence,"Thursday, December 29",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,"4,949",Kadence,"Thursday, December 29",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
4,"9,900",Kadence,"Thursday, December 29",https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...
17015,None,Yamaha,None,None
17016,None,GUITAR,None,None
17017,None,Mustang,None,None
17018,None,RK,None,None


In [8]:
import numpy as np #importing numpy
p_df4 =p_df3.replace(np.nan, '-') #replace nan with "-"
p_df4.isnull().sum() #count of null is 0

Price                0
Brand Name           0
Expected Delivery    0
Prouduct link        0
dtype: int64

In [9]:
p_df4

,Price,Brand Name,Expected Delivery,Prouduct link
0,"6,299",Kadence,"Tuesday, December 27",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,"9,900",Kadence,"Thursday, December 29",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,"4,949",Kadence,"Thursday, December 29",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,"4,949",Kadence,"Thursday, December 29",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
4,"9,900",Kadence,"Thursday, December 29",https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...
17015,-,Yamaha,-,-
17016,-,GUITAR,-,-
17017,-,Mustang,-,-
17018,-,RK,-,-


In [10]:
amz_driver.close()

3. Write a python program to access the search bar and search button on images.google.com and scrape 10
images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [11]:
import urllib # We will require urllib to download images

google_driver = webdriver.Chrome(r"D:\chromedriver.exe")
google_driver.maximize_window()

In [12]:
google_driver.get("https://google.com") # get the google website
time.sleep(1)
#lets click on images button to land on images.google.com
imgs = google_driver.find_element(By.XPATH, "/html/body/div[1]/div[1]/div/div/div/div[1]/div/div[2]/a")
imgs.click()
time.sleep(2)

In [13]:
#Scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

search_bar = google_driver.find_element(By.XPATH, "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input")
#/html/body/c-wiz/c-wiz/div/div[3]/div[2]/div/div[1]/form/div[1]/div[2]/div/div[2]/input
submit_button = google_driver.find_element(By.XPATH, "//span[@class = 'z1asCe MZy1Rb']")


In [14]:
search_bar.send_keys('fruits')
submit_button.click()
time.sleep(2)

Fruits = []
f_img = google_driver.find_elements(By.XPATH, "//img[contains(@class, 'rg_i Q4LuWd')]")
for i in f_img[0:10]:
    Fruits.append(i.get_attribute('src'))
    if len(Fruits) == 10:
        break
print(len(Fruits))
for i in range (10):
    urllib.request.urlretrieve(str(Fruits[i]),"Downloads/assignment3/fruits/fruits{}.jpg".format(i))


10


In [15]:
Search_bar = google_driver.find_element(By.XPATH, "/html/body/c-wiz/c-wiz/div/div[3]/div[2]/div/div[1]/form/div[1]/div[2]/div/div[2]/input")
Search_bar.clear()
submit_button = google_driver.find_element(By.XPATH, "//span[@class = 'n6h3Rc']")
Search_bar.send_keys('Car')
submit_button.click()
time.sleep(2)

Cars = []

c_img = google_driver.find_elements(By.XPATH, "//img[contains(@class, 'rg_i Q4LuWd')]")
for i in c_img[0:10]:
    Cars.append(i.get_attribute('src'))
    if len(Cars) == 10:
        break
print(len(Cars))
for i in range(10):
    urllib.request.urlretrieve(str(Cars[i]),"Downloads/assignment3/Cars/cars{}.jpg".format(i))
    

    
#10 images has been downloaded
    


10


In [16]:
search_bar = google_driver.find_element(By.XPATH, "/html/body/c-wiz/c-wiz/div/div[3]/div[2]/div/div[1]/form/div[1]/div[2]/div/div[2]/input")
search_bar.clear()
submit_button = google_driver.find_element(By.XPATH, "//span[@class = 'n6h3Rc']")
search_bar.send_keys("Machine Learning")
submit_button.click()


ML = []

machine_learning = google_driver.find_elements(By.XPATH,"//img[contains(@class, 'rg_i Q4LuWd')]")
for i in machine_learning[0:10]:
    ML.append(i.get_attribute('src'))
    if len(ML) == 10:
        break
print(len(ML))
for i in range(0,10):
    urllib.request.urlretrieve(str(ML[i]),"Downloads/assignment3/ML/ML{}.jpg".format(i))
    
#10 ML jpg file downloaded


10


In [17]:
search_bar  = google_driver.find_element(By.XPATH, "/html/body/c-wiz/c-wiz/div/div[3]/div[2]/div/div[1]/form/div[1]/div[2]/div/div[2]/input")
search_bar.clear()
submit_button = google_driver.find_element(By.XPATH, "//span[@class = 'n6h3Rc']")
search_bar.send_keys("Cake")
submit_button.click()

cake = []
cakes = google_driver.find_elements(By.XPATH,"//img[contains(@class, 'rg_i Q4LuWd')]")
for i in cakes[0:10]:
    cake.append(i.get_attribute('src'))
print(len(cake))

for i in range(0,10):
    urllib.request.urlretrieve(str(cake[i]), "Downloads/assignment3/cake/cake{}.jpg".format(i))
    
#10 images has been downloaded
    

10


In [18]:
google_driver.close()

4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on
www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be 
scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, 
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the 
details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [19]:
flip_driver = webdriver.Chrome("r/D:\chromedriver.exe")
flip_driver.maximize_window()
flip_driver.get("https://www.flipkart.com/")
time.sleep(3)
close_registration = flip_driver.find_element(By.XPATH, "/html/body/div[2]/div/div/button")
close_registration.click() #closing the registration window appear in the starting 

In [20]:
search_bar = flip_driver.find_element(By.XPATH, "/html/body/div[1]/div/div[1]/div[1]/div[2]/div[2]/form/div/div/input")
submit_button = flip_driver.find_element(By.XPATH, "//button[@class = 'L0Z3Pu']")
search_bar.send_keys("Oneplus Nord")
submit_button.click()
time.sleep(2)

In [21]:
brand = []
smart_phone = []
brand_names = flip_driver.find_elements(By.XPATH, "//div[@class = '_4rR01T']")
for i in brand_names:
    brand.append(i.text.split(" ")[0])#.split(" ")
    
#print(brand)

for i in brand_names:
    smart_phone.append(i.text.split(" ")[1])
#print(smart_phone)


In [22]:
colour = [] #emepty list to store firstplit data
colour_name = [] #empty list to store final split data
for i in brand_names:
    colours = i.text.split("(")[1]
    colour.append(colours)
for i in colour:
    colour_name.append(i.split(",")[0])
#print(colour_name)


In [23]:
ROM = [] # RAM list to store RAM data
for i in brand_names:
    rom = i.text.split(",")[1]
    ROM.append(rom.split(")")[0] + " " + "ROM")
#print(ROM)

In [24]:
RAM = [] #ROM list to store ROM list
other_info = flip_driver.find_elements(By.XPATH, "//ul[@class = '_1xgFaf']")
for i in other_info:
    RAM.append(i.text.split("|")[0])
#print(RAM)

In [25]:
camera = []

for i in other_info:
    camera.append(i.text.split("\n")[2])
#print(camera)

In [26]:
display = []
for i in other_info:
    display.append(i.text.split("\n")[1])
#print(display)

In [27]:
Battery = []
for i in other_info:
    Battery.append(i.text.split("\n")[3])
#print(Battery)

In [28]:
price = []
prices = flip_driver.find_elements(By.XPATH, "//div[@class = '_30jeq3 _1_WHN1']")
for i in prices:
    price.append(i.text)
#print(price)

In [29]:
P_URL = []
urls = flip_driver.find_elements(By.XPATH, "//a[@class = '_1fQZEK']")
for i in urls:
    P_URL.append(i.get_attribute('href'))
#print(P_URL)

In [30]:
#Data Frame for all the fetched details

flip_df = pd.DataFrame({"Brand Name": brand, "Smart Phone Name":smart_phone, "Colour":colour_name, "RAM":RAM, "ROM":ROM, "Camera":camera, "Display Size":display, "Battery":Battery, "Price":price, "Product link":P_URL})
flip_df                        

,Brand Name,Smart Phone Name,Colour,RAM,ROM,Camera,Display Size,Battery,Price,Product link
0,OnePlus,Nord,Blue Tide,6 GB RAM,128 GB ROM,64MP Rear Camera,16.74 cm (6.59 inch) Display,5000 mAh Battery,"₹19,879",https://www.flipkart.com/oneplus-nord-ce-2-lit...
1,OnePlus,Nord,Black Dusk,6 GB RAM,128 GB ROM,64MP Rear Camera,16.74 cm (6.59 inch) Display,5000 mAh Battery,"₹19,970",https://www.flipkart.com/oneplus-nord-ce-2-lit...
2,OnePlus,Nord,Blue Tide,6 GB RAM,128 GB ROM,64MP Rear Camera,16.74 cm (6.59 inch) Display,5000 mAh Battery,"₹19,957",https://www.flipkart.com/oneplus-nord-ce-2-lit...
3,OnePlus,Nord,Black Dusk,6 GB RAM,128 MB ROM,64MP Rear Camera,16.74 cm (6.59 inch) Display,5000 mAh Battery,"₹19,895",https://www.flipkart.com/oneplus-nord-ce-2-lit...
4,OnePlus,Nord,Blue Tide,6 GB RAM,128 GB ROM,64MP Rear Camera | 16MP Front Camera,16.74 cm (6.59 inch) HD+ Display,5000 mAh Battery,"₹19,999",https://www.flipkart.com/oneplus-nord-ce-2-lit...
5,OnePlus,Nord,Black Dusk,6 GB RAM,128 GB ROM,64MP Rear Camera,16.74 cm (6.59 inch) Display,5000 mAh Battery,"₹19,888",https://www.flipkart.com/oneplus-nord-ce-2-lit...
6,OnePlus,Nord,Jade Fog,8 GB RAM,128 GB ROM,50MP Rear Camera,17.02 cm (6.7 inch) Display,4500 mAh Battery,"₹28,999",https://www.flipkart.com/oneplus-nord-2t-5g-ja...
7,OnePlus,Nord,Bahama Blue,8 GB RAM,128 GB ROM,64MP Rear Camera,16.33 cm (6.43 inch) Display,4500 mAh Battery,"₹24,998",https://www.flipkart.com/oneplus-nord-ce-2-5g-...
8,OnePlus,Nord,Charcoal Ink,6 GB RAM,128 GB ROM,64MP Rear Camera,16.33 cm (6.43 inch) Display,4500 mAh Battery,"₹20,899",https://www.flipkart.com/oneplus-nord-ce-5g-ch...
9,OnePlus,Nord,PAC-MAN,12 GB RAM,256 GB ROM,50MP Rear Camera,16.33 cm (6.43 inch) Display,4500 mAh Battery,"₹37,999",https://www.flipkart.com/oneplus-nord-2-5g-pac...


In [31]:
#flip_df.to_csv(r"D:\Amir Ali\Data Science Backup\FlipKart Smartphone.csv")
#csv file has been created at the given location

In [32]:
flip_driver.close()

5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google
maps.

In [33]:
gmap = webdriver.Chrome(r"D:\chromedriver.exe")
gmap.maximize_window()
gmap.get("https://www.google.com/maps")
time.sleep(2)


In [34]:
search_bar = gmap.find_element(By.XPATH, "/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div/div[2]/form/input[1]")
search_bar.send_keys("Delhi")
time.sleep(1)
submit_button = gmap.find_element(By.XPATH, "/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div/div[2]/div[1]/button")
submit_button.click()
time.sleep(2)

In [35]:
#As suggested on doubt clearing session we can use current url

url_string = gmap.current_url
url = url_string.split("@")[1]
print("latitude, longitude are = ", url.split(",")[0], ",", url.split(",")[1])

latitude, longitude are =  27.1416735 , 80.8833819


In [36]:
gmap.close()

6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) 
from trak.in

In [37]:
trak_driver = webdriver.Chrome("D:\chromedriver.exe")
trak_driver.maximize_window()

In [38]:
# menu_click = trak_driver.find_element(By.XPATH, "/html/body/header/div[2]/div/div/a[1]")
# menu_click.click()

In [39]:
trak_driver.get("https://trak.in/stories/")
time.sleep(2)
funding_deal = trak_driver.find_element(By.XPATH, "/html/body/header/div[3]/div/div/div[1]/nav/div[2]/div/ul/li[9]/a")
funding_deal.click()
time.sleep(1)

#There is no data available inside "Funding Deals

In [40]:
trak_driver.close()

7. Write a program to scrap all the available details of best gaming laptops from digit.in

In [41]:
digit_driver = webdriver.Chrome("D:\chromedriver.exe")
digit_driver.maximize_window()

In [42]:
digit_driver.get("https://www.digit.in/")
time.sleep(2)

In [43]:
laptops = digit_driver.find_element(By.XPATH, "/html/body/div[1]/div/div[4]/ul/li[3]/a")
laptops.click()
time.sleep(1)

In [44]:
gaming_laptop = digit_driver.find_element(By.XPATH, "/html/body/div[6]/div/div[2]/div[2]/ul/li[10]/a")
gaming_laptop.click()
time.sleep(2)

In [45]:
laptop_name = []
laptopnames = digit_driver.find_elements(By.XPATH, "//div[@class = 'left_side']/a")
for i in laptopnames:
    laptop_name.append(i.text)
print(laptop_name)

['MSI Titan GT77-12UHS', 'Alienware X17 R2', 'Acer Predator Triton 500 SE PT516-52s', 'Omen By HP (16-B1371TX)', 'Acer Predator Helios 300 AN515-45', 'MSI Delta 15 (A5EFK-083IN)', 'Omen By HP (16-C0141AX)', 'Lenovo Legion 5i Pro (82RF00MGIN)', 'Alienware M15 R5 Ryzen Edition Icc-C780001win', 'Lenovo Slim 7 Gen 6 (82K8002JIN)']


In [46]:
available_details = []

other_info = digit_driver.find_elements(By.XPATH, "//div[@class = 'Wrap-pop']")

for i in other_info:
    available_details.append(i.text)
print(available_details)

['Windows 11 Home OS', '17.3" (3840 x 2160) Display', '12th Gen Intel Core i9-12900HX | NA Processor', '2 TB SSD/64 GB DDR5 Memory', 'Windows 11 Home OS', '17.3" (1920 x 1080) Display', '12th Gen Intel Core i9-12900H | 5 GHz Processor', '1 TB SSD/32 GB DDR5 Memory', 'Windows 11 OS', '16" (2560 x 1600) Display', '12th Gen Intel Core i7-12700H | 3.5 GHz Processor', '2 TB SSD/32 GB DDR5 Memory', 'Windows 11 Home OS', '16.1" (2560 x 1440) Display', '12th Gen Intel Core i7-12700H | 4.7 GHz Processor', '1 TB SSD/8 GB DDR5 Memory', 'Windows 11 Home OS', '15.6" (2560 x 1440) Display', 'AMD Ryzen 9-5900HX | NA Processor', '512 GB SSD/16 GB DDR4 Memory', 'Windows 11 Home OS', '15.6" (1920 x 1080) Display', 'AMD 5th Gen Ryzen 9-5900HX | 3.3GHz Processor', '1 TB SSD/16 GBGB DDR4 Memory', 'Windows 11 Home OS', '16.1" (2560 x 1440) Display', 'AMD Ryzen™ 9 5900HX | 3.3 GHz Processor', '1 TB NVMe/16 GB DDR4 Memory', 'Windows 11 Home OS', '16" (2560 x 1600) Display', '12th Gen Intel Core i7-12700H | 2.

In [47]:
digit_driver.close()

8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be 
scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [48]:
forbes_driver = webdriver.Chrome("D:\chromedriver.exe")
forbes_driver.maximize_window()

In [49]:
forbes_driver.get("https://www.forbes.com/")
time.sleep(5)

In [50]:
menu = forbes_driver.find_element(By.XPATH, "//div[@class = '_69hVhdY4']")
menu.click()
time.sleep(3)

In [51]:
viewall_b = forbes_driver.find_element(By.XPATH, "/html/body/div[1]/header/nav/div[1]/div/div/div[2]/ul/li[1]/div[1]")
viewall_b.click()
time.sleep(2)

In [52]:
world_billion = forbes_driver.find_element(By.XPATH, "/html/body/div[1]/main/div/section/section[1]/div/div/div[1]/div/div[1]/div[1]/div[2]")
world_billion.click()
time.sleep(3)



In [53]:
# singup_close = forbes_driver.find_element(By.XPATH, "/html/body/div[8]/div/button")
# singup_close.click()

In [54]:
rank = []
ranks = forbes_driver.find_elements(By.XPATH, "//div[@class = 'rank']")
for i in ranks:
    rank.append(i.text)
#print(rank)

In [55]:
name = []
names = forbes_driver.find_elements(By.XPATH, "//div[@class = 'personName']")
for i in names:
    name.append(i.text)
#print(name)

In [56]:
networth = []
net_worth = forbes_driver.find_elements(By.XPATH, "//div[@class = 'netWorth']")
for i in net_worth:
    networth.append(i.text)
#print(networth)

In [57]:
age = []
ages = forbes_driver.find_elements(By.XPATH, "//div[@class = 'age']/div")
for i in ages:
    #print(i.text)
    age.append(i.text)
#print(age)

#/html/body/div[1]/div[1]/div/div[2]/div[3]/div[2]/div[2]/div[2]/div[1]/div[2]/div[1]/div[4]/div

In [58]:
citizenship = []
citizenships = forbes_driver.find_elements(By.XPATH, "//div[@class = 'countryOfCitizenship']")
for i in citizenships:
    citizenship.append(i.text)
#print(citizenship)

In [59]:
source = []
sources = forbes_driver.find_elements(By.XPATH, "//span[@class = 'source-text']")
for i in sources:
    source.append(i.text)
#print(source)

In [60]:
industry = []
industries = forbes_driver.find_elements(By.XPATH, "//div[@class = 'category']/div")
for i in industries:
    industry.append(i.text)
#print(industry)

In [61]:
df = pd.DataFrame({"Rank":rank, "Name":name, "Networth": networth, "Age":age, "Cizinship":citizenship, "Income Source": source, "industry": industry})
df

,Rank,Name,Networth,Age,Cizinship,Income Source,industry
0,1.,Elon Musk,$219 B,50,United States,"Tesla, SpaceX",Automotive
1,2.,Jeff Bezos,$171 B,58,United States,Amazon,Technology
2,3.,Bernard Arnault & family,$158 B,73,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$129 B,66,United States,Microsoft,Technology
4,5.,Warren Buffett,$118 B,91,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
195,192.,Marcel Herrmann Telles,$10.3 B,72,Brazil,beer,Food & Beverage
196,197.,Leon Black,$10 B,70,United States,private equity,Finance & Investments
197,197.,Joe Gebbia,$10 B,40,United States,Airbnb,Technology
198,197.,David Geffen,$10 B,79,United States,"movies, record labels",Media & Entertainment


In [62]:
forbes_driver.close()

9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted 
from any YouTube Video

In [63]:
yt_driver = webdriver.Chrome("D:\chromedriver.exe")
yt_driver.maximize_window()

In [64]:
yt_driver.get("https://www.youtube.com/")
time.sleep(5)

In [65]:
search_bar = yt_driver.find_element(By.XPATH, "/html/body/ytd-app/div[1]/div/ytd-masthead/div[3]/div[2]/ytd-searchbox/form/div[1]/div[1]/input")
search_bar.send_keys("how to lodge complaint on pg portal")
time.sleep(2)

submit_button = yt_driver.find_element(By.XPATH, "/html/body/ytd-app/div[1]/div/ytd-masthead/div[3]/div[2]/ytd-searchbox/button")
submit_button.click()
time.sleep(10)


In [66]:
play_video = yt_driver.find_element(By.XPATH, "/html/body/ytd-app/div[1]/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div[2]/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-video-renderer[1]/div[1]/ytd-thumbnail")
play_video.click()
time.sleep(10)

In [67]:
# scroll_pause_time = 1 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
# screen_height = yt_driver.execute_script("return window.screen.height;")   # get the screen height of the web
# i = 1

# while True:
#     # scroll one screen height each time
#     yt_driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
#     i += 1
#     time.sleep(scroll_pause_time)
#     # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
#     scroll_height = yt_driver.execute_script("return document.body.scrollHeight;")  
#     # Break the loop when the height we need to scroll to is larger than the total scroll height

In [68]:
for _ in range(0, 1000):
    yt_driver.execute_script("window.scrollBy(0,1000)")
    

In [69]:
comment = []
comments = yt_driver.find_elements(By.ID, "content-text")
for i in comments[0:500]:
    comment.append(i.text)
print(comment)
    

[]


In [70]:
print(len(comment))

0


In [71]:
yt_driver.close()

10.Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in 
“London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, 
overall reviews, privates from price, dorms from price, facilities and property description

In [72]:
hostel_driver = webdriver.Chrome("D:\chrmoedriver.exe")
hostel_driver.maximize_window()

In [73]:
hostel_driver.get("https://www.hostelworld.com/")
time.sleep(5)

In [74]:
icon = hostel_driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div[2]/div[2]/div/div/div[4]/div/div[2]/div/div[1]/div/input")
icon.click()

In [75]:
search_bar = hostel_driver.find_element(By.XPATH, "//div[@class = 'search']/input")
#search_bar.click()
#time.sleep(1)
search_bar.send_keys("London")
time.sleep(3)

In [76]:
loc_search = hostel_driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div[2]/div[2]/div/div/div[4]/div/div[2]/div/div[1]/div/div/ul/li[2]")
loc_search.click()

In [77]:
submit_button = hostel_driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div[2]/div[2]/div/div/div[4]/div/div[2]/div/div[5]/button")
submit_button.click()
time.sleep(5)

In [78]:
hostel_name = []
hostelnames = hostel_driver.find_elements(By.XPATH, "//h2[@class = 'title title-6']")
for i in hostelnames:
    hostel_name.append(i.text)
print(hostel_name)

["Wombat's City Hostel London", "St Christopher's Village", 'Hostel One Notting Hill', 'Urbany Hostel London', 'Generator London', 'Safestay London Elephant & Castle', 'Smart Russell Square Hostel', "St Christopher's Camden", 'No.8 Seven Sisters', 'No.8 Willesden Hostel London', 'Smart Hyde Park Inn Hostel', 'Safestay London Kensington Holland Park', 'NX London Hostel', 'Astor Museum Inn', 'Smart Camden Inn Hostel', 'Phoenix Hostel', "St Christopher's Hammersmith", 'Smart Hyde Park View Hostel', 'Queen Elizabeth Chelsea', 'Astor Victoria', 'Prime Backpackers Angel', 'International Students House', 'Saint James Backpackers', "St Christopher's Greenwich", 'Clink261', "St Christopher's Shepherds Bush", 'PubLove @ The Steam Engine, Waterloo', 'Strand Continental', 'PubLove @ The White Ferry, Victoria', 'Book a Bed Hostels']


In [79]:
#Distance from central City
distance = []
distance_cc = hostel_driver.find_elements(By.XPATH, "//div[@class = 'subtitle body-3']")
for i in distance_cc:
    distance.append(i.text.split("-")[1])
print(distance)

[' 3.6km from city centre', ' 1.8km from city centre', ' 3.6km from city centre Show on Map', ' 1.8km from city centre Show on Map', ' 5.5km from city centre Show on Map', ' 5.4km from city centre Show on Map', ' 3km from city centre Show on Map', ' 1.7km from city centre Show on Map', ' 2.6km from city centre Show on Map', ' 4.3km from city centre Show on Map', ' 9km from city centre Show on Map', ' 10km from city centre Show on Map', ' 5km from city centre Show on Map', ' 5.9km from city centre Show on Map', ' 6.1km from city centre Show on Map', ' 2.2km from city centre Show on Map', ' 4.4km from city centre Show on Map', ' 4.2km from city centre Show on Map', ' 7.5km from city centre Show on Map', ' 5km from city centre Show on Map', ' 5.7km from city centre Show on Map', ' 1.8km from city centre Show on Map', ' 3.6km from city centre Show on Map', ' 3.3km from city centre Show on Map', ' 5.5km from city centre Show on Map', ' 7.6km from city centre Show on Map', ' 3.2km from city 

In [80]:
#ratting
ratting = []
review_type = []
total_reviews = []
h_ratting = hostel_driver.find_elements(By.XPATH, "//div[@class = 'bottom-rating']")
for i in h_ratting:
    ratting.append(i.text.split("\n")[0])
    review_type.append(i.text.split("\n")[1])
    total_reviews.append(i.text.split("\n")[2])
print(ratting)
print(review_type)
print(total_reviews)



['8.9', '8.0', '9.3', '9.2', '7.7', '7.3', '7.1', '8.2', '5.4', '6.8', '7.5', '7.0', '7.5', '8.0', '8.4', '7.0', '7.7', '7.9', '7.1', '7.8', '8.2', '9.0', '7.5', '7.2', '7.9', '7.1', '8.1', '6.9', '7.7', '5.9']
['Fabulous', 'Fabulous', 'Superb', 'Superb', 'Very Good', 'Very Good', 'Very Good', 'Fabulous', 'Rating', 'Good', 'Very Good', 'Very Good', 'Very Good', 'Fabulous', 'Fabulous', 'Very Good', 'Very Good', 'Very Good', 'Very Good', 'Very Good', 'Fabulous', 'Superb', 'Very Good', 'Very Good', 'Very Good', 'Very Good', 'Fabulous', 'Good', 'Very Good', 'Rating']
['14419 Total Reviews', '11768 Total Reviews', '1638 Total Reviews', '577 Total Reviews', '7296 Total Reviews', '4735 Total Reviews', '9848 Total Reviews', '3931 Total Reviews', '3930 Total Reviews', '4914 Total Reviews', '6452 Total Reviews', '1401 Total Reviews', '1422 Total Reviews', '9135 Total Reviews', '3001 Total Reviews', '3905 Total Reviews', '4175 Total Reviews', '4648 Total Reviews', '3371 Total Reviews', '14867 Tot

In [81]:
#price
drom_forms = []
d_price = hostel_driver.find_elements(By.XPATH, "//div[@class = 'price-col']")
for i in d_price:
    drom_forms.append(i.text.split("\n"))
    form_price.append(i.text.split("\n"))
#print(drom_forms)

NameError: name 'form_price' is not defined

In [ ]:
df =pd.DataFrame(drom_forms)
df.columns = ["Type", "Price"]
df

In [ ]:
#facilities and property description
facilities_in_h = []
facilities = hostel_driver.find_elements(By.XPATH, "//div[@class = 'facilities-label facilities']")
for i in facilities:
    facilities_in_h.append(i.text)
facilities_in_h
    

In [ ]:
hostel_driver.close()